In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting

import matplotlib.pyplot as plt
import seaborn as sns
# Configs
%config InlineBackend.figure_format ='retina'
%matplotlib inline

import time

from sklearn import base, neural_network, neighbors, feature_extraction, datasets, linear_model, metrics, model_selection, preprocessing, svm, ensemble, decomposition, feature_extraction, utils


In [1]:
# Load Bridge helper classes
%run load-data.ipynb

data = load_deals(1000)

In [ ]:
# Concatenating features
X = [np.array([h.feature_array() for h in d.hands]).flatten()
     for d in data]

# Winners in a particular strain played from N
y = [d.wnrs[strains.index('NT')]['N'] for d in data]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, PReLU

def seq_model(input_s, layers_no=4, layers_s=256, activation='relu', optimizer='sgd', dropout=False, loss='mean_squared_error',
              **kwargs):
    model = Sequential()
    # Dense(64) is a fully-connected layer with 64 hidden units.
    # in the first layer, you must specify the expected input data shape:
    # here, 20-dimensional vectors.
    model.add(Dense(layers_s, activation=activation, input_dim=input_s, **kwargs))
    for _ in range(0, layers_no-1):
        if dropout:
            model.add(Dropout(0.5))
        model.add(Dense(layers_s, activation=activation, **kwargs))

    model.add(Dense(1, activation='relu', **kwargs))
    model.compile(loss=loss, optimizer=optimizer, metrics=['mean_absolute_error'])
    return model

In [ ]:
def train_eval(models,
               features=[(is_NS, '52-vector with 0/1 if it belongs to NS')], 
               targets=[(imp_diff_900_6NT, 'IMP-difference for playing 6NT vs 900-score')]):
    
    if not isinstance(models, list):
        models = [models]
    
    for X, f_desc in features:
        print ("Features: " + f_desc + "\n")
        for y, t_desc in targets:
            print ("    Targets: " + t_desc)
            ts = time.time(); accuracies = []
            X_tr, X_ts, y_tr, y_ts = model_selection.train_test_split(X, y, test_size=0.2, random_state=1)
            
            for m_d in models:
                if 'desc' not in m_d:
                    m_d['desc'] = 'unknown m '
                for kw in 'bfp':
                    if kw+'_args' not in m_d:
                        m_d[kw+'_args'] = {}
                
                if isinstance(m_d['m'], base.BaseEstimator):
                    m = m_d['m']
                else:
                    # Model has to be built
                    m = m_d['m'](input_s=X_tr.shape[1], **m_d['b_args'])
                
                ts = time.time()
                m.fit(X_tr, y_tr, **m_d['f_args'])
                print ("        M: " + m_d['desc'] + " # ", end='')
                y_pred = m.predict(X_ts, **m_d['p_args'])
                y_tr_pred = m.predict(X_tr, **m_d['p_args'])
                #print ("P:",  y_pred, end='')
                print (" test MAE: " + "{0:.2f}".format(metrics.mean_absolute_error(y_ts, y_pred)), end='' )
                print (" - tr MAE: " + "{0:.2f}".format(metrics.mean_absolute_error(y_tr, y_tr_pred)), end='' )
                print (" T: " + "{0:.1f}".format(time.time() - ts) + "s")
                

In [ ]:
train_eval({'m': linear_model.LinearRegression(), 'desc': 'LinearRegr'})  # 1 min on GPU

In [ ]:
train_eval({'m': seq_model, 'desc': 'MLPNadamTan',
            'b_args': {'activation': 'tanh', 'optimizer': keras.optimizers.Nadam()},
            'f_args': {'batch_size': 32, 'epochs': 4, 'verbose': 2}})  # 1 min on GPU